# Video Metadata

The project involved multiple cameras, each of which may have moved or used different exposure settings, and the method of synchronization changed across the project. It is therefore hard to keep track of what video tracking data is available and when. This notebook aims to provide a guide to help us understand what is available and for when.

Let's start by getting the details of all available videos from the project database:

In [4]:
%matplotlib inline

import os, sys
from pathlib import Path

from dotenv import load_dotenv

import matplotlib.pyplot as plt
import pandas as pd

load_dotenv()

sys.path.insert(0, str(Path.cwd().parent.parent.parent))
from lib import utils

In [5]:
videos = utils.query_postgres("SELECT * FROM task_switch.video_files;")
videos.head()

,session_dt,ferret,block,filename,frame_count,frame_width,frame_height,default_fps,tdt_samps,rv2,webcam
0,2016-03-23 09:26:30.438,1506,J1-17,2016-03-23_Track_09-26-31.avi,18378.0,640.0,480.0,30.0,False,False,True
1,2016-03-23 09:26:30.438,1506,J1-17,F1506_Phoenix_Block_J1-17_Vid0.avi,55530.0,640.0,480.0,30.0,False,True,False
2,2016-03-23 17:44:56.668,1506,J1-19,2016-03-23_Track_17-44-57.avi,23724.0,640.0,480.0,30.0,False,False,True
3,2016-03-23 17:44:56.668,1506,J1-19,F1506_Phoenix_Block_J1-19_Vid0.avi,71656.0,640.0,480.0,30.0,False,True,False
4,2016-03-24 09:05:20.391,1506,J1-20,2016-03-24_Track_09-05-20.avi,33576.0,640.0,480.0,30.0,False,False,True


Each session may have multiple videos - we first want to split the data into those sessions that have an RV2 file. I'm sure the code below could be improved, but it's all I can figure out at the moment in 30C with no AC...

In [ ]:
has_RV2, no_RV2 = [],[]

for session, s_data in videos.groupby('session_dt'):

    if any(s_data['rv2']):
        has_RV2.append(s_data)
    else:
        no_RV2.append(s_data)

has_RV2 = pd.concat(has_RV2)
no_RV2 = pd.concat(no_RV2)

print(f"Original data ({videos.shape[0]} rows)")
print(f"Data for sessions with RV2 videos ({has_RV2.shape[0]} rows)")
print(f"Data for sessions without RV2 videos ({no_RV2.shape[0]} rows)")

In [ ]:
no_RV2.plot.scatter(x='session_dt', y='ferret')
plt.yticks(no_RV2.ferret.unique())
plt.show()

Look at the relative numbers of blocks for each ferret:

In [ ]:
def count_sessions(df, colname):

    return (
        df
        .groupby('ferret')
        .agg({'block':['count']})
        .rename({'count':colname}, axis=1)
    )

pd.concat([
    count_sessions(has_RV2[has_RV2.rv2], 'has_RV2'),
    count_sessions(no_RV2, 'no_RV2')
], axis=1)

## 1. Sessions without RV2 videos

There are a small number of blocks for which there is no RV2 video, usually because the file was corrupted during recording. 

The cell below shows those videos for F1506_Phoenix, for whome the videos in Block J2-37 and J2-44 might be recoverable.

In [ ]:
no_RV2[no_RV2.ferret == 1506]

In other cases, such as F1517, all that is available in the blocks without RV2 data are the secondary videos for which frame rates are much lower. It's not clear how viable it would be to build a separate synchronization pipeline for these videos when they are so rare.

In [ ]:
no_RV2.query("ferret == 1517")

In [ ]:
no_RV2.query("ferret == 1518")

In [ ]:
no_RV2.query("ferret == 1602")

What about the later ferrets

In [ ]:
no_RV2.query("ferret == 1605 and not tdt_samps")

In [ ]:
no_RV2.query("ferret == 1605 and tdt_samps")

## 2. Videos without visual signals for synchronization

The majority of videos are synchronized using the times of visual stimuli to identify dropped frames and estimate ongoing frame rate. However this isn't possible for some sessions in which only auditory stimuli were presented. Let's get a list of those blocks for which only auditory trials were presented, and link that to the relevant RV2 videos here:


In [19]:
query = """ 
WITH CTE AS (
SELECT 
	ss.ferret,
	ss.block,
	ss.datetime as session_dt,
	SUM( CASE WHEN ii.modality = 'Auditory' THEN 1 ELSE 0 END) as n_auditory,
	COUNT(*) as n_trials
FROM task_switch.sessions ss
INNER JOIN task_switch.trials tt
	ON ss.datetime = tt.session_dt
INNER JOIN task_switch.stimuli ii
	ON tt.stim_id = ii.id
GROUP BY ss.datetime)

SELECT 
	vf.ferret, vf.block, session_dt, n_auditory, n_trials, default_fps,	vf.filename
FROM task_switch.video_files vf
INNER JOIN (
	SELECT * 
	FROM CTE 
	WHERE n_auditory = n_trials 
		AND block IS NOT NULL) aud_sessions
	USING(session_dt)
WHERE rv2 is true
ORDER BY vf.ferret, vf.block;"""

sound_only_sessions = utils.query_postgres(query)

print(f"Auditory only sessions: n = {sound_only_sessions.shape[0]}")
sound_only_sessions.head()

Auditory only sessions: n = 46


,ferret,block,session_dt,n_auditory,n_trials,default_fps,filename
0,1506,J1-38,2016-03-28 10:17:51.028,82,82,30.0,F1506_Phoenix_Block_J1-38_Vid0.avi
1,1518,J2-6,2016-04-12 08:39:20.258,26,26,30.0,F1518_Rita_Block_J2-6_Vid0.avi
2,1518,J2-7,2016-04-12 17:07:38.024,53,53,30.0,F1518_Rita_Block_J2-7_Vid0.avi
3,1602,J2-9,2016-09-28 12:41:19.461,2,2,11.0,F1602_Agatha_Block_J2-9_Vid0.avi
4,1605,J2-52,2017-04-13 10:26:51.696,31,31,30.0,F1605_Snorlax_Block_J2-52_Vid0.avi


For these blocks, we will have to approximate frame times using a fixed frame rate. We know in advance that the frame rates associated with each video file  are 

In [24]:
data_dir = Path(os.getenv("local_home"))
ongoing_fps_dir = data_dir / 'Task_Switching/head_tracking/ongoing_fps'

# Load fps estimate 
ongoing_fps = []
for fps_file in ongoing_fps_dir.glob('*.csv'):

    ongoing_fps.append(
        pd.read_csv(fps_file, usecols=['n_frames','fps'])
    )

ongoing_fps = pd.concat(ongoing_fps)

In [31]:
avg_fps = sum(ongoing_fps['fps'] * ongoing_fps['n_frames']) / ongoing_fps['n_frames'].sum()
print(f"Grand averege fps = {avg_fps}")

Grand averege fps = 29.866522783879137


## 3. Directly-synchronized videos

These are videos in which video capture script included a call to the other data acquisition system (TDT) to request it's clock time (in samples from the start of the recording block) for every frame. Although subject to some delays, this approach has a better ceiling on temporal alignment than would be possible with a flashing LED. (The best solution would be a frame counter that increments based on a digital output signal).

Calibration images have not been associated with these videos yet.

The metadata for these files looks off - the frame width and height are 

### Sessions with timestamps only in images

It may be possible to recover these using [pytesseract](https://pypi.org/project/pytesseract/) using [get_tdt_samp_images.py](./../synchronization/get_tdt_samp_images.py) and [clean_OCR_estimates.py](./../synchronization/clean_OCR_estimates.py) (see [ReadMe.md](./../ReadMe.md) for more info).

 Some videos did not save correctly:
* F1605 Block_J5-15 

 TDT samples were logged incorrectly as all zeros for:
* F1605 Block_J5-16

In [ ]:
no_RV2.query("'2018-01-29 00:00:00.000' < session_dt < '2018-02-20 00:00:00.000'")

### Sessions after which TDT samples were logged

Note: TDT samples were logged incorrectly as all zeros for:
* F1605 Block_J5-32 
* F1613 Block_J5-24

The rest look initially as would be expected.

In [ ]:
no_RV2.query("session_dt > '2018-02-20 00:00:00.000'")

## Tracking Metadata

RV2 videos fall into two categories based on image exposure, with separate deeplabcut models used for tracking LEDs (low exposure) and LEDs + other landmarks (head, nose, shoulders, hips and tail; high exposure). 

The tracking for videos with direct synchronization was done using a separate deeplabcut model (FrontiersLED_HEmat-Sisyphus-2022-09-01) that was based on a different image size.

In [ ]:
from dotenv import load_dotenv
from pathlib import Path
import os

load_dotenv()

file_path = Path(os.getenv('local_home')) / 'Task_Switching/head_tracking'
file_names = {
    "high_exposure" : 'DLC_data_high_230213_0322.parquet',
    "low_exposure" : 'DLC_data_low_230213_0227.parquet'
}

In [ ]:
from collections import Counter
import itertools
import pyarrow.parquet as pq

# Load schemas from each file
schemas = dict()

for file_type, file_name in file_names.items():
    schemas[file_type] = pq.read_schema( file_path / file_name)

# Tell us how many times is each feature available
Counter(
    itertools.chain(
        *[schemas[k].names for k in schemas.keys()]
        )
    )

In [ ]:
# List all blocks
import pandas as pd
tracking = []

for file_type, file_name in file_names.items():
    print(f"Loading {file_name} - may take a second...")
    
    temp = pq.read_table( file_path / file_name, columns=['fnum','block'])
    temp = temp.to_pandas().drop_duplicates()
    temp['file'] = file_type[0]

    tracking.append(temp)

tracking = pd.concat(tracking)
print(tracking.shape)

Are all RV2 video files accounted for? No, we haven't considered how to process low exposure videos when the animal wasn't recorded from, and therefore has no LEDs.

In [ ]:
# Select blocks for which tracking isn't in the input data
is_RV2 = videos[videos.rv2]

rv2_tracking = pd.merge(
    left = is_RV2[['ferret','block','filename']],
    right = tracking,
    how = 'left',
    left_on = ['ferret','block'],
    right_on = ['fnum','block']
)

if rv2_tracking['file'].isna().sum() == 0:
    print(f"All RV2 files (n = {rv2_tracking.shape[0]}) accounted for")
else:
    rv2_tracking.query("file.isna()").sort_values(['ferret','block'])

### Tracking results for videos that aren't part of the project

Are there any videos in the tracking data that aren't included in the videos associated with the project.

This can happen, for example when a video has been recorded for a short period in which the animals didn't do any trials and the session was stopped - usually this was related to a technical issue in which another block with a new number was started.

Notes:
Sessions without videos: F1607 Block_J3-12

In [ ]:
video_blocks = videos[['ferret','block']].drop_duplicates()
video_blocks.shape